![SAP Ariba logo](./images/sap-ariba-small.jpeg)  

# Asynchronous Reporting API - Submit and process async job requests

## Overview

When retrieving large amounts of reporting data, it is recommended to use the async functionality available in the reporting APIs. To achieve this, we first need to create a job, monitor the job status and once the job is in status `completed` we can then proceed to retrieve the files generated for our request. The example shown here applies for both, the Analytical and Operational Reporting APIs. The only difference will be in the URL called but the process is the same. The process is documented here: https://help.sap.com/viewer/c4f46b9331834a0b970f834c20c9c73b/latest/en-US/78cdc1ef7ab74201ba26615c999bfc76.html.

| API | Documentation | Comments |
| :-- | :-- | :-- |
| Operational Reporting API for Strategic Sourcing | https://help.sap.com/viewer/c4f46b9331834a0b970f834c20c9c73b/latest/en-US |  |
| Analytical Reporting API for Strategic Procurement and Operational Procurement | https://help.sap.com/viewer/bf0cde439a0142fbbaf511bfac5b594d/cloud/en-US |  |

## Step 0 - Initialise script

Define functions that will be used by our script and authentication method

In [1]:
import json
import os
import zipfile

import requests

def get_access_token(base64_auth):
    payload={}
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Basic {base64_auth}'
    }

    response = requests.request("POST", OAUTH_API_URL, headers=headers, data=payload)

    return response.json()['access_token']


### Setup API details

<div class="alert alert-warning">
⚠️ When running locally, you will need to substitute the details below with your own application details for the script to work. The API key, Base64 values included here will not grant you access to a realm.
</div>

In [2]:
# Authentication server
OAUTH_API_URL = 'https://api.ariba.com/v2/oauth/token?grant_type=client_credentials'

REALM = 'MYREALM-T'

# Operational Reporting API
OPER_REPORTING_ASYNC_BASEURL = 'https://openapi.ariba.com/api/sourcing-reporting-job/v1/prod'
OPER_REPORTING_ASYNC_JOB_RESULT_URL = 'https://openapi.ariba.com/api/sourcing-reporting-jobresult/v1/prod'
OPER_REPORTING_APIKEY = 'uEnCwXMo7YYmQE7el7iqqciAqT7Og0Ik'
OPER_REPORTING_BASE64 = 'M0jVjWIlRzTEYZ04UMTjFNU1iITHCUZpEQONOmTdpXiFxWNlA0lVSRDQ2OMtZJ1ajBV2GjZEdTkLwMlYGOcImHHtpxjF'

In [3]:
# Get access tokens

OPER_REPORTING_TOKEN = get_access_token(OPER_REPORTING_BASE64)
OPER_REPORTING_TOKEN

'2239053e-fd69-4073-9a9b-5a735fa2de01'

## Step 1 - Submit Job Asynchronously

<div class="alert alert-warning">
⚠️ In this example we will retrieve ProjectAuditInfo from the Operational Reporting API, we need to define previously a view template of document type ProjectAuditInfo.
</div>

In the code block below, we submit an asynchronous job to the Operational Reporting API. We specify the view template defined for the ProjectAuditInfo document type, e.g. ProjectAuditInfo_SpendOverview and overwrite the default filters set in the view template, see values in payload.

**API call:**
<div class="alert alert-info">
    <b>POST</b> https://openapi.ariba.com/api/sourcing-reporting-job/v1/prod/jobs?realm=MYREALM-T
</div>

In [4]:
# View template previously defined. The document type is ProjectAuditInfo
view_template = 'ProjectAuditInfo_SpendOverview'

date_from = '2021-03-22T00:00:00Z'
date_to = '2021-03-22T11:59:59Z'

url = f"{OPER_REPORTING_ASYNC_BASEURL}/jobs?realm={REALM}"
        
payload = {
  "viewTemplateName": view_template,
  "filters": {
    "updatedDateFrom": date_from,
    "updatedDateTo": date_to
  }
}

headers = {
  'Content-Type': 'application/json',
  'apikey': OPER_REPORTING_APIKEY,
  'Authorization': f'Bearer {OPER_REPORTING_TOKEN}'
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
submit_job_response = response.json()

# Display results
print(f"POST {url}")
print()
print(json.dumps(submit_job_response, indent=2))

POST https://openapi.ariba.com/api/sourcing-reporting-job/v1/prod/jobs?realm=MYREALM-T

{
  "jobId": "c04a7d9a-7394-4431-b509-9e503fd450a51616410714984",
  "files": [],
  "status": "pending",
  "createdDate": "2021-03-22T10:58:34Z",
  "completedDate": null,
  "viewTemplateName": "ProjectAuditInfo_SpendOverview",
  "filters": {
    "updatedDateFrom": "2021-03-22T00:00:00Z",
    "entryType": [
      2,
      8
    ],
    "updatedDateTo": "2021-03-22T11:59:59Z"
  },
  "documentType": "ProjectAuditInfo",
  "selectAttributesSnap": [
    "EffectiveUser",
    "Parameters",
    "LineId",
    "NodeName",
    "ResourceName",
    "Changes.FieldName",
    "Changes.BaseObjectClass",
    "ContextObject",
    "Changes.ValueClass",
    "TimeUpdated",
    "TimeStamp",
    "ResourceKey",
    "TimeCreated",
    "EntryType",
    "IsSystem",
    "Level",
    "Id",
    "ContextVersion",
    "RealUser"
  ],
  "filterExpressionsSnap": [
    {
      "name": "createdDateTo",
      "field": "TimeCreated",
      

In [5]:
job_id = submit_job_response['jobId']
job_id

'c04a7d9a-7394-4431-b509-9e503fd450a51616410714984'

## Step 2 - Get job status

After some time, we can check the status of the job. A job can be in the following statuses: `pending, processing, completed`. Now, we proceed to check the status of the job we just submitted.

<div class="alert alert-info">
<b>GET</b> https://openapi.ariba.com/api/sourcing-reporting-jobresult/v1/prod/jobs/c04a7d9a-7394-4431-b509-9e503fd450a51616410714984?realm=MYREALM-T
</div>

In [6]:
url = f"{OPER_REPORTING_ASYNC_JOB_RESULT_URL}/jobs/{job_id}?realm={REALM}"
                
headers = {
  'apikey': OPER_REPORTING_APIKEY,
  'Authorization': f'Bearer {OPER_REPORTING_TOKEN}'
}

response = requests.request("GET", url, headers=headers)
job_status_response = response.json()

# Display results
print(f"GET {url}")
print()
print(json.dumps(response.json(), indent=2))


GET https://openapi.ariba.com/api/sourcing-reporting-jobresult/v1/prod/jobs/c04a7d9a-7394-4431-b509-9e503fd450a51616410714984?realm=MYREALM-T

{
  "jobId": "c04a7d9a-7394-4431-b509-9e503fd450a51616410714984",
  "files": [
    "Fkmkh87x2.zip"
  ],
  "status": "completed",
  "createdDate": "2021-03-22T10:58:34Z",
  "completedDate": "2021-03-22T10:59:14Z",
  "viewTemplateName": "ProjectAuditInfo_SpendOverview",
  "filters": {
    "updatedDateFrom": "2021-03-22T00:00:00Z",
    "entryType": [
      2,
      8
    ],
    "updatedDateTo": "2021-03-22T11:59:59Z"
  },
  "documentType": "ProjectAuditInfo",
  "selectAttributesSnap": [
    "EffectiveUser",
    "Parameters",
    "LineId",
    "NodeName",
    "ResourceName",
    "Changes.FieldName",
    "Changes.BaseObjectClass",
    "ContextObject",
    "Changes.ValueClass",
    "TimeUpdated",
    "TimeStamp",
    "ResourceKey",
    "TimeCreated",
    "EntryType",
    "IsSystem",
    "Level",
    "Id",
    "ContextVersion",
    "RealUser"
  ],
  "f

The job status is completed. We can now retrieve the files containing the data that was requested.

In [7]:
files = job_status_response['files']
files

['Fkmkh87x2.zip']

## Step 3 - Get files generated for job

A job can contain multiple zip files - `files` array. Our script needs to be capable of retrieving each file and extract the data within the zip file.

<div class="alert alert-warning">
⚠️ Depending on the number of records that the request might be able to retrieve, it is possible that the <code>job_status_response</code> payload contains a pageToken node. If so, you will need to submit an additional job specifying the page token to retrieve subsequent records, e.g. <b>POST</b> https://openapi.ariba.com/api/sourcing-reporting-job/v1/prod/jobs?realm=MYREALM-T&pageToken=ABCD1234XYZ 
</div>

<div class="alert alert-info">
    <b>GET</b> https://openapi.ariba.com/api/sourcing-reporting-jobresult/v1/prod/jobs/c04a7d9a-7394-4431-b509-9e503fd450a51616410714984/files/Fkmkh87x2.zip?realm=MYREALM-T
</div>

In [9]:
for file_id in files:

    url = f"{OPER_REPORTING_ASYNC_JOB_RESULT_URL}/jobs/{job_id}/files/{file_id}?realm={REALM}"

    headers = {
      'apikey': OPER_REPORTING_APIKEY,
      'Authorization': f'Bearer {OPER_REPORTING_TOKEN}'
    }

    response = requests.request("GET", url, headers=headers)
    
    # Display results
    print(f"GET {url}")
    print()
    
    # Save zip file
    zip_filename = f"./files/{file_id}"
    json_filename = zip_filename.replace('zip','json')

    with open(zip_filename, "wb") as binary_file: 
        # Write bytes to file 
        binary_file.write(response.content)

    # Extract zip file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(".")

        os.system(f"mv records.txt {json_filename}")

    # Prepare payload
    with open(json_filename, 'r') as json_file:
        contents = json_file.read()
        
        # Wrapping the contents in a Records node to easily process the results in Python
        payload = json.loads('{ "Records": ' + contents + ' }')

        print(json.dumps(payload, indent=2))

GET https://openapi.ariba.com/api/sourcing-reporting-jobresult/v1/prod/jobs/c04a7d9a-7394-4431-b509-9e503fd450a51616410714984/files/Fkmkh87x2.zip?realm=MYREALM-T

{
  "Records": [
    {
      "EffectiveUser": {
        "FirstName": null,
        "Phone": "",
        "LastName": null,
        "Fax": "",
        "MiddleName": null,
        "UniqueName": "aribasystem",
        "EmailAddress": "no-reply@smtp.ariba.com",
        "Name": "aribasystem"
      },
      "Changes": null,
      "Parameters": {
        "Param10": "{String}Jii2AJYB2a8X3w",
        "Param9": "{String}4",
        "Param8": "",
        "Param7": "",
        "Param6": "",
        "Param5": "",
        "Param4": "",
        "Param3": "",
        "Param2": "",
        "Param1": "{String}1100002212 - 4100001752"
      },
      "LineId": "c1rsf7:T39",
      "NodeName": "C2_TaskCXML1",
      "ResourceName": "ariba.collaborate.audit",
      "ContextObject": {
        "InternalId": "SR37986071"
      },
      "TimeUpdated": "2